# CTR Study


This dataset has been taken from <a href="https://www.kaggle.com/c/kddcup2012-track2/overview">Kaggle</a>

Search advertising has been one of the major revenue sources of the Internet industry for years. A key technology behind search advertising is to predict the click-through rate (pCTR) of ads, as the economic model behind search advertising requires pCTR values to rank ads and to price clicks. In this task, given the training instances derived from session logs of the Tencent proprietary search engine, soso.com, participants are expected to accurately predict the pCTR of ads in the testing instances.


**TRAINING DATA FILE**

The training data file is a text file, where each line is a training instance derived from search session log messages. To understand the training data, let us begin with a description of search sessions.   

A search session refers to an interaction between a user and the search engine. It contains the following ingredients: the user, the query issued by the user, some ads returned by the search engine and thus impressed (displayed) to the user, and zero or more ads that were clicked by the user. For clarity, we introduce a terminology here. The number of ads impressed in a session is known as the ’depth’. The order of an ad in the impression list is known as the ‘position’ of that ad. An Ad, when impressed, would be displayed as a short text known as ’title’, followed by a slightly longer text known as the ’description’, and a URL (usually shortened to save screen space) known as ’display URL’.   

We divide each session into multiple instances, where each instance describes an impressed ad under a certain setting  (i.e., with certain depth and position values).  We aggregate instances with the same user id, ad id, query, and setting in order to reduce the dataset size. Therefore, schematically, each instance contains at least the following information:

UserID 
AdID 
Query 
Depth 
Position 
Impression 
the number of search sessions in which the ad (AdID) was impressed by the user (UserID) who issued the query (Query).

Click 
the number of times, among the above impressions, the user (UserID) clicked the ad (AdID).   

Moreover, the training, validation and testing data contain more information than the above list, because each ad and each user have some additional properties. We include some of these properties into the training, validation  and the testing instances, and put other properties in separate data files that can be indexed using ids in the instances. For more information about these data files, please refer to the section ADDITIONAL DATA FILES. 


**Columns**

Finally, after including additional features, each training instance is a line consisting of fields delimited by a whitespace character: 

1. Click: as described in the above list. 

2. Impression: as described in the above list. 

3. DisplayURL: a property of the ad. The URL is shown together with the title and description of an ad. It is usually the shortened landing page URL of the ad, but not always. In the data file,  this URL is hashed for anonymity. 

4. AdID: as described in the above list. 

5. AdvertiserID: a property of the ad. Some advertisers consistently optimize their ads, so the title and description of their ads are more attractive than those of others’ ads. 

6. Depth: a property of the session, as described above.   

7. Position: a property of an ad in a session, as described above. 

8. QueryID:  id of the query. This id is a zero‐based integer value. It is the key of the data file 'queryid_tokensid.txt'.

9. KeywordID: a property of ads. This is the key of  'purchasedkeyword_tokensid.txt'. 

10. TitleID: a property of ads. This is the key of 'titleid_tokensid.txt'. 

11. DescriptionID: a property of ads. This is the key of 'descriptionid_tokensid.txt'. 

12. UserID: This is the key of 'userid_profile.txt'. When we cannot identify the user, this field has a special value of 0.


**ADDITIONAL DATA FILES**

*(Not yet provided but you can download it from Kaggle)*

There are five additional data files, as mentioned in the above section: 

1. userid_profile.txt 

Each line of ‘userid_profile.txt’ is composed of UserID, Gender, and Age, delimited by the TAB character. Note that not every UserID in the training and the testing set will be present in ‘userid_profile.txt’. Each field is described below: 

1. Gender: 

'1'  for male, '2' for female,  and '0'  for unknown. 

2. Age: 

'1'  for (0, 12],  '2' for (12, 18], '3' for (18, 24], '4'  for  (24, 30], '5' for (30,  40], and '6' for greater than 40. 

#### Spark session

In [ ]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import IntegerType, StructType, StructField

In [ ]:
# Spark Context
spark = SparkSession.builder.appName("ctr").config("spark.sql.caseSensitive", "True").getOrCreate()

## Study

In [7]:
clicksDF = spark.read.option("inferSchema", "true")\
                .option("header", "false")\
                .option("sep", " ")\
                .csv("gs://ucmbucket/data/ctr.txt")\
                .toDF("Click", "Impression", "DisplayURL", "AdID", "AdvertiserID", "Depth",
                      "Position", "QueryID", "KeywordID", "TitleID", "DescriptionID", "UserID")\
           .cache()
clicksDF.show()

+-----+----------+--------------------+--------+------------+-----+--------+--------+---------+-------+-------------+------+
|Click|Impression|          DisplayURL|    AdID|AdvertiserID|Depth|Position| QueryID|KeywordID|TitleID|DescriptionID|UserID|
+-----+----------+--------------------+--------+------------+-----+--------+--------+---------+-------+-------------+------+
|    0|         1| 4298118681424644510| 7686695|         385|    3|       3|    1601|     5521|   7709|          576|490234|
|    0|         1| 4860571499428580850|21560664|       37484|    2|       2| 2255103|      317|  48989|        44771|490234|
|    0|         1| 9704320783495875564|21748480|       36759|    3|       3| 4532751|    60721| 685038|        29681|490234|
|    0|         1|13677630321509009335| 3517124|       23778|    3|       1|    1601|     2155|   1207|         1422|490234|
|    0|         1| 3284760244799604489|20758093|       34535|    1|       1| 4532751|    77819| 266618|       222223|490234|


<div class="alert alert-block alert-success">
<b>QUESTIONS</b>:
    <ul>
        <li>¿Cuántos usuarios distintos y anuncios distintos hay? ¿Cuántos anunciantes?             
        <li> ¿Cuál es el número medio de anuncios distintos visualizados por cada usuario?
        <li> ¿Cuál es el número medio de anunciantes vistos por un usuario?
        <li> Calcular el CTR (Click-through rate) para cada usuario como el número total de clicks del usuario entre el número total de impresiones para ese usuario.
        <li> En esa operación, incluir también el recuento del número de anuncios vistos por el usuario. ¿Tiene relación con el CTR?
        <li> Utilizando el dataset de la edad y el grupo de los usuarios, obtener el CTR medio de cada grupo de usuarios a partir de una agregación en el grupo de todos los CTR individuales de los usuarios de ese grupo. 
    </ul>
</div>

* ¿Cuántos usuarios distintos y anuncios distintos hay? ¿Cuántos anunciantes?         

In [9]:
unicosDF = clicksDF.select(F.countDistinct("UserID").alias("usuarios"),
               F.countDistinct("AdID").alias("anuncios"),
               F.countDistinct("AdvertiserID").alias("anunciantes"))

unicosDF.show()

+--------+--------+-----------+
|usuarios|anuncios|anunciantes|
+--------+--------+-----------+
|   13310|   20014|       5590|
+--------+--------+-----------+



* ¿Cuál es el número medio de anuncios distintos visualizados por cada usuario?
* ¿Cuál es el número medio de anunciantes vistos por un usuario?

In [12]:
# Aprovecho este groupBy para calcular el resultado intermedio que necesito para después calcular cada media global
intermediosDF = clicksDF.groupBy("UserID").agg(
    F.countDistinct("AdID").alias("anunciosDist"),
    F.countDistinct("AdvertiserID").alias("anunciantesDist"),
    (F.sum("Click") / F.sum("Impression")).alias("CTR"),   # aprovecho para resolver también la siguiente pregunta de CTR
    F.count("*").alias("conteo")
    ).cache()

intermediosDF.select(
            F.mean("anunciosDist").alias("mediaAnunciosDist"),
            F.mean("anunciantesDist").alias("mediaAnunciantesDist")
        ).show()

+-----------------+--------------------+
|mediaAnunciosDist|mediaAnunciantesDist|
+-----------------+--------------------+
|3.815777610818933|   3.321111945905334|
+-----------------+--------------------+



* Calcular el CTR (Click-through rate) para cada usuario como el número total de clicks del usuario entre el número total de impresiones para ese usuario.
* En esa operación, incluir también el recuento del número de anuncios vistos por el usuario. ¿Tiene relación con el CTR?

In [14]:
# Calculamos la correlación entre el recuento y el CTR
intermediosDF.select(F.corr("CTR", "conteo").alias("correlacion CTR y conteo")).show()

+------------------------+
|correlacion CTR y conteo|
+------------------------+
|    -0.02528239618792...|
+------------------------+



Prácticamente no hay correlación entre ambos, lo cual significa que no por mostrar muchos anuncios a un usuario, su tasa de clicks  por anuncio visto (CTR) va a ser mayor (sí podría serlo su total acumulado de clicks). El CTR va más bien asociado a la personalidad del usuario y en todo caso, al grupo de edad.

* Utilizando el dataset de la edad y el grupo de los usuarios, obtener el **CTR medio de cada grupo de edad y sexo** a partir de una agregación de los CTR individuales de los usuarios de cada grupo. 

In [23]:
profileSchema = StructType([
    StructField("UserID", IntegerType(), True),
    StructField("gender", IntegerType(), True),
    StructField("age", IntegerType(), True)
])

userID_profile = spark.read.option("header", "false")\
                           .option("sep", "\t")\
                           .schema(profileSchema)\
                           .csv("gs://ucmbucket/data/userid_profile.txt").cache()

userID_profile.printSchema()
userID_profile.show(5)

root
 |-- userID: integer (nullable = true)
 |-- gender: integer (nullable = true)
 |-- age: integer (nullable = true)

+------+------+---+
|userID|gender|age|
+------+------+---+
|     1|     1|  5|
|     2|     2|  3|
|     3|     1|  5|
|     4|     1|  3|
|     5|     2|  1|
+------+------+---+
only showing top 5 rows



In [ ]:
# Usamos pivot para visualizar mejor la comparación entre CTR de hombres y mujeres en cada grupo
# Esto tarda un rato
ctrGruposDF = intermediosDF.join(userID_profile, "UserID")\
             .groupBy("age")\
             .pivot("gender")\
             .agg(F.mean("CTR").alias("CTR medio")).cache()

In [26]:
# Según la descripción del dataset: 
#   gender:  1' for male, '2' for female, and '0' for unknown
#   age: '1' for (0, 12], '2' for (12, 18], '3' for (18, 24], '4' for (24, 30], '5' for (30, 40], and '6' for greater than 40
ctrGruposDF.withColumnRenamed("1", "Hombres")\
           .withColumnRenamed("2", "Mujeres")\
           .withColumnRenamed("0", "Caracol")\
           .withColumn("age", F.when(F.col("age") == 1, "(0, 12]")
                               .when(F.col("age") == 2, "(12, 18]")
                               .when(F.col("age") == 3, "(18, 24]")
                               .when(F.col("age") == 4, "(24, 30]")
                               .when(F.col("age") == 5, "(30, 40]")
                               .when(F.col("age") == 6, ">40")
                      )\
            .sort("age").show()

+--------+--------------------+--------------------+--------------------+
|     age|             Caracol|             Hombres|             Mujeres|
+--------+--------------------+--------------------+--------------------+
| (0, 12]|0.056460804940534674|  0.0588565910355278|0.053055667256233466|
|(12, 18]| 0.03571428571428571| 0.06182935850474643| 0.04730646322441077|
|(18, 24]| 0.08235294117647059|0.059243100033103714| 0.05420430682528766|
|(24, 30]| 0.03571428571428571| 0.05648487718545309| 0.06543772003700819|
|(30, 40]|                0.25| 0.04995657985024774|0.060111061538502236|
|     >40| 0.17708333333333331|  0.0633456855722357| 0.05417440727298718|
+--------+--------------------+--------------------+--------------------+

